# Проект по SQL. Сервис для чтения книг по подписке. 

В рамках исследования нужно проанализировать базу данных, которая содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Дано 5 таблицы: books(данные о книгах), authors(данные об авторах), publishers(данные об издательствах), ratings(данные о пользовательских оценках книг), reviews(данные о пользовательских обзорах).

Требуется:
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Предпросмотр и предобработка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books'''
con=engine.connect()
books = pd.io.sql.read_sql(sql=text(query), con = con)
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [2]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [3]:
books['publication_date'] = pd.to_datetime(books['publication_date'])

In [4]:
books.duplicated().sum()
#дубликатов нет

0

In [5]:
books['title'].nunique()

999

In [6]:
query = '''SELECT * FROM authors'''
con=engine.connect()
authors = pd.io.sql.read_sql(sql=text(query), con = con)
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [7]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [8]:
authors.duplicated().sum()
#дубликатов нет

0

In [9]:
query = '''SELECT * FROM publishers'''
con=engine.connect()
publishers = pd.io.sql.read_sql(sql=text(query), con = con)
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [10]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [11]:
publishers.duplicated().sum()
#дубликатов нет

0

In [12]:
query = '''SELECT * FROM ratings'''
con=engine.connect()
ratings = pd.io.sql.read_sql(sql=text(query), con = con)
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [13]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [14]:
ratings.duplicated().sum()
#дубликатов нет

0

In [15]:
query = '''SELECT * FROM reviews'''
con=engine.connect()
reviews = pd.io.sql.read_sql(sql=text(query), con = con)
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


In [16]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [17]:
reviews.duplicated().sum()
#дубликатов нет

0

## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года 

In [18]:
query = '''SELECT COUNT(book_id) 
           FROM books 
           WHERE EXTRACT(YEAR FROM CAST (publication_date AS timestamp)) >= 2000
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


После 1 января 2000 года вышло 821 книга

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [19]:
query = '''SELECT b.book_id,
                  b.title,
                  COUNT(DISTINCT r.review_id) num_of_reviews,
                  ROUND (AVG(rt.rating), 2) avg_rating
           FROM books b
           INNER JOIN ratings rt ON b.book_id = rt.book_id
           LEFT JOIN reviews r ON b.book_id = r.book_id
           GROUP BY b.book_id, b.title
           ORDER BY COUNT(r.review_id) DESC
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,num_of_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,750,The Hobbit or There and Back Again,6,4.13
2,673,The Catcher in the Rye,6,3.83
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Больше всего ревью было сделано на книгу Twilight (Twilight #1) - в два раза больше чем, на книгу The Hobbit or There and Back Again, которая занимает 2 место

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [20]:
query = '''SELECT p.publisher,
                  COUNT(b.book_id)
           FROM publishers p
           JOIN books b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY COUNT(b.book_id) DESC
           LIMIT 5
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Издательством, которое выпустило наибольшее число книг толще 50 страниц стало Penguin Books

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [21]:
query = '''SELECT author,
                  ROUND (AVG(rt.rating), 2) avg_rating,
                  COUNT(rt.rating_id) num_of_estimations
           FROM authors a
           LEFT JOIN books b ON a.author_id = b.author_id
           LEFT JOIN ratings rt ON b.book_id = rt.book_id
           WHERE b.book_id IN
                           (SELECT b.book_id
                            FROM books b
                            JOIN ratings rt ON b.book_id=rt.book_id
                            GROUP BY b.book_id
                            HAVING COUNT(rt.rating_id)>=50)
           GROUP BY author
           ORDER BY AVG(rt.rating) DESC
           LIMIT 5
           '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating,num_of_estimations
0,J.K. Rowling/Mary GrandPré,4.29,310
1,Markus Zusak/Cao Xuân Việt Khương,4.26,53
2,J.R.R. Tolkien,4.25,162
3,Louisa May Alcott,4.19,52
4,Rick Riordan,4.08,62


Авторы с самой высокой средней оценкой книг - это J.K. Rowling и Mary GrandPré

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок 

In [22]:
query ='''SELECT AVG(users_cnt) 
          FROM (SELECT COUNT(r.review_id) users_cnt
          FROM reviews r 
          WHERE username IN (SELECT rt.username
          FROM ratings rt
          GROUP BY rt.username
          HAVING COUNT(rt.username) > 48)
          GROUP BY username) rt
          '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равняется 24

## Вывод

В рамках исследования бл проведен анализ базы данных, которая содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Ответы на основные вопросы исследования дали понять, что:
1. В базе хранится 821 книга, которые вышли после 1 января 2000 года
2. Больше всего обозревали следующие книги: Twilight (Twilight #1), The Hobbit or There and Back Again, The Catcher in the Rye
3. Издательством, которое выпустило наибольшее число книг толще 50 страниц стало Penguin Books. Книги с количеством страниц меньше 50 являются брошюрами 
4. Автор с самой высокой средней оценкой книг - это Diana Gabaldon, его оценка - 4,3
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равняется 24

Рекомендации бизнесу:
1. Стоит увеличить выбор книг, предстваленных в сервисе.
2. Имеет смысл выкладывать на сервисе книги авторов с самыми высокими оценками
3. Можно сделать специальные предложения с продажей книг, у которых есть несколько частей